In [9]:
import duckdb
import pandas as pd
import plotly.express as px
from plotly_calplot import calplot

from common.env_variables import DUCKDB_DWH_FILE


def display_df(_df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False,
                           "display.float_format", '${:,.2f}'.format):
        display(_df.fillna('.'))


def display_sql(sql_statement):
    conn = duckdb.connect(DUCKDB_DWH_FILE, read_only=True)
    _df = conn.execute(sql_statement).df()
    display_df(_df)
    conn.close()
    return _df


In [10]:
_ = display_sql(f'''
SELECT job_id,
       COUNT(1) AS total
  FROM curated.job_description
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 5
''')

,job_id,total
0,7543521,12
1,7369771,10
2,7723680,9
3,7599993,8
4,7571802,8


In [11]:
_ = display_sql(f'''
SELECT load_timestamp,
       title,
       online_date,
       job_hashdiff
  FROM curated.job_description
 WHERE job_id = 7543521
 ORDER BY load_timestamp DESC
 LIMIT 20
''')

,load_timestamp,title,online_date,job_hashdiff
0,2022-01-26 16:26:20,Vertriebsmitarbeiter/in Innendienst (m/w/d),2022-01-02T13:03:06Z,b55e16942b367490a89e79a2b5e5877ede96dddb845550...
1,2022-01-20 10:00:00,Innendienst Vertrieb Ausstellung (m/w/d),2022-01-02T13:03:06Z,23c8455292f8232cae2ef032bfcde68065eb54e3f4a403...
2,2022-01-10 19:00:00,Mitarbeiter/in Vertrieb Ausstellung (m/w/d),2022-01-02T13:03:06Z,558e6978d9474422c1aa11c1b3a2e9f35f80471394b7e0...
3,2022-01-08 13:00:00,Berater Ausstellung (m/w/d),2022-01-02T13:03:06Z,d14b662ed262e04fb43262a4e3c2ee45d4d757942d525d...
4,2021-12-18 14:00:00,Verkaufsberater Ausstellung (m/w/d),2021-12-18T13:03:05Z,f89da56d3db440f3211df54f149910ead6010b6c1faa4b...
5,2021-11-20 11:00:00,Berater Ausstellung (m/w/d),2021-11-13T17:03:10Z,c7e4b24bc45578706d3084ca1aa13d87587e51cdfc3843...
6,2021-11-12 16:00:00,Fachberater Ausstellung (m/w/d),2021-10-29T15:30:01Z,2e0925e3f4a375a9d7107ed4aa93988778e217867715c8...
7,2021-11-10 17:00:00,Fachberater - Glaser / Schreiner (m/w/d),2021-10-29T15:30:01Z,19d9b5c5a368417bf3f8fb92a6a4afabfa8c72068493cc...
8,2021-10-14 21:00:00,Kaufmännische/r Angestellte/r (m/w/d),2021-10-06T15:03:04Z,2834598b4c6db613f5b817a63f3ca1f24d7c255934d780...
9,2021-10-07 08:00:00,Kaufmännische/r Angestellte/r (m/w/d),2021-10-06T15:03:04Z,2834598b4c6db613f5b817a63f3ca1f24d7c255934d780...


In [16]:
_ = display_sql(f'''
SELECT job_id,
       load_timestamp
FROM (
    SELECT d.*,
           row_number()
           OVER (
            PARTITION BY job_id ORDER BY load_timestamp DESC
           ) AS seqnum
      FROM curated.job_description d
     WHERE job_id = 7543521
) d
WHERE seqnum = 1;
''')

,job_id,load_timestamp,title,online_status,is_anonymous,should_display_early_applicant,contract_type,work_type,online_date,company_name,description_introduction,description_responsabilities,description_requirements,description_perks,job_hashdiff,day,month,year,seqnum
0,7543521,2022-01-26 16:26:20,Vertriebsmitarbeiter/in Innendienst (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2022-01-02T13:03:06Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,b55e16942b367490a89e79a2b5e5877ede96dddb845550...,26,1,2022,1
1,7543521,2022-01-20 10:00:00,Innendienst Vertrieb Ausstellung (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2022-01-02T13:03:06Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,23c8455292f8232cae2ef032bfcde68065eb54e3f4a403...,20,1,2022,2
2,7543521,2022-01-10 19:00:00,Mitarbeiter/in Vertrieb Ausstellung (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2022-01-02T13:03:06Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,558e6978d9474422c1aa11c1b3a2e9f35f80471394b7e0...,10,1,2022,3
3,7543521,2022-01-08 13:00:00,Berater Ausstellung (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2022-01-02T13:03:06Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,d14b662ed262e04fb43262a4e3c2ee45d4d757942d525d...,8,1,2022,4
4,7543521,2021-12-18 14:00:00,Verkaufsberater Ausstellung (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2021-12-18T13:03:05Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,f89da56d3db440f3211df54f149910ead6010b6c1faa4b...,18,12,2021,5
5,7543521,2021-11-20 11:00:00,Berater Ausstellung (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2021-11-13T17:03:10Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,c7e4b24bc45578706d3084ca1aa13d87587e51cdfc3843...,20,11,2021,6
6,7543521,2021-11-12 16:00:00,Fachberater Ausstellung (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2021-10-29T15:30:01Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,2e0925e3f4a375a9d7107ed4aa93988778e217867715c8...,12,11,2021,7
7,7543521,2021-11-10 17:00:00,Fachberater - Glaser / Schreiner (m/w/d),online,False,False,Feste Anstellung,Vollzeit,2021-10-29T15:30:01Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,19d9b5c5a368417bf3f8fb92a6a4afabfa8c72068493cc...,10,11,2021,8
8,7543521,2021-10-14 21:00:00,Kaufmännische/r Angestellte/r (m/w/d),online,False,True,Feste Anstellung,Vollzeit,2021-10-06T15:03:04Z,Götz Management GmbH,EinleitungUnser Auftraggeber ist mit ca. 400 M...,Beratung der Besucher und Kunden\nBesucherempf...,abgeschlossene handwerkliche oder kaufmännisch...,unbefristete Anstellung und hohe Arbeitsplatzs...,2834598b4c6db613f5b817a63f3ca1f24d7c255934d780...

In [ ]:
_ = display_sql(f'''
WITH () AS

SELECT job_id,
       load_timestamp
FROM (
    SELECT d.*,
           row_number()
           OVER (
            PARTITION BY job_id ORDER BY load_timestamp DESC
           ) AS seqnum
      FROM curated.job_description d
     WHERE job_id = 7543521
) d
WHERE seqnum = 1;
''')
